Automatic differentation allows us to calculate potentially millions of partial derivatives of virtually any function with only one extra pass of code.

Goal; To implement 4 methods of calculating European put option price and Greeks. Automatic differentiation can be used to calculate option price and greeks. Methods are; 

1. Black scholes formula and closed form equations for using option greeks.
2. Calculate greeks using finite differences. 
3. Use black scholes formulas for option values and use Pytorch to automatically differentiate the output and calculate the option greeks. 
4. Use monte Carlo simulations of option values in Pytorch. 

More information found; https://www.jtash.com/black-scholes-in-pytorch

In [ ]:
# Pytorch runs on fast hardware accelerators using GPU's and computational graphs are createdto fly as oppoised to static graphs. 
# Start inputs to our formulas as tensors and start gradient tracking for those inputs. 
# Once we have our output we can run reverse mode automatic differentation to calculate gradient of the output with respect to the input 
import torch 
import autograd

In [ ]:
# Black scholes inputs 
stock = torch.tensor(100.0, requires_grad = True) # stock price of the option S
strike = torch.tensor(100.0 , requires_grad = True) # strike price
rate = torch.tensor(0.035, requires_grad = True)  # normally denoted r but in articale it is u
vol = torch.tensor(0.16, requires_grad = True) # annualized volatility 
time = torch.tensor(1.0, requires_grad = True) # time expiry in years
dividend = torch.tensor(0.01, requires_grad = True) # dividend

# utility functions 
cdf = torch.distributions.Normal(0,1).cdf # calling normal cdf of mean 0 and standard deviation 1
pdf = lambda x: torch.distributions.Normal(0,1).log_prob(x).exp()


In [ ]:
# Main thing 
# Create samples such that ;
'''
The four variables make a sample x is a data point x = (t, S, r, sigma) is data points. 
No stable results -> then normalise the varaibles. 
       t -> (0,2) _ 0,1,2 only discrete number of days / 365. T is measure in years. 
       S -> (50,150) stock prices. 
       r -> [0%,1%] 
       sigma (volatility) -> [1%,30%] -> standard deviation of the log of the returns

'''

In [ ]:
# P = k*e**(-u*t)*N(-d2)-S*e**(-delta*t)*N(-d1)
# d1 = (ln(S/K) + (u-delta + sigma**2/2)*t)/ sigma * root(t)
# d2 = ln(S/K) + (u - delta - sigma**2/2)*t / sigma * root(t)

In [ ]:
# Method 1

In [ ]:
#  Black Scholes option value and greeks 

d1 = (torch.log(stock/strike)+(rate-dividend + vol*vol/2)*time)/ vol * (time**0.5)
d2 = (torch.log(stock/strike)+(rate-dividend - vol*vol/2)*time)/ vol * (time**0.5)
P = strike* torch.exp(-rate*time)*cdf(-d2) - stock*torch.exp(-dividend*time)*cdf(-d1)

delta = torch.exp(-dividend*time) * (cdf(d1) - 1) # delta is derivative of option wrt to stock
rho = -strike * time * torch.exp(-rate*time) * cdf(-d2) # derivative of option with respect to rate ( u)
vega = stock * torch.exp(-dividend*time) * pdf(d1) * time**0.5 # derivtive of option with respect to volatilty 
theta = -(
    rate * strike * torch.exp(-rate*time) * cdf(-d2)
    - stock * torch.exp(-dividend*time) * pdf(d1) * vol / (2*time**0.5) 
    - dividend * stock * torch.exp(-dividend*time) * cdf(-d1)
) # derivtive of option with respect to time maturity product rule on d1
epsilon = stock * time * torch.exp(-dividend*time) * cdf(-d1) #  derivative of option with respect to derivative
strike_greek = torch.exp(-rate*time) * cdf(-d2)


In [ ]:
print(P) # option price value is 5.09

tensor(5.0885, grad_fn=<SubBackward0>)


In [ ]:
# if security S increases from 100 to 101 then delta will decrease from -0.4026 to 0.40
# imcreasing price of an option increases theta
# decease time of matruity deceases greeks sensitivity of theta


In [ ]:
print(delta)
print(rho)
print(vega)
print(theta)
print(epsilon)
print(strike_greek)

tensor(-0.4026, grad_fn=<MulBackward0>)
tensor(-45.3458, grad_fn=<MulBackward0>)
tensor(38.4103, grad_fn=<MulBackward0>)
tensor(1.8883, grad_fn=<NegBackward>)
tensor(40.2573, grad_fn=<MulBackward0>)
tensor(0.4535, grad_fn=<MulBackward0>)


In [ ]:
#Method 2 finite differnces - bump and revalue

# f'(x) =_ (f(x+ delta) - f(x))/ delta - provides estimate of the partial derivative
# black-scholes formula
strike = 100

def bs(stock, rate, vol, time, dividend):
    d1 = (torch.log(stock/strike) + (rate - dividend + vol*vol/2)*time) / (vol*time**0.5)
    d2 = (torch.log(stock/strike) + (rate - dividend - vol*vol/2)*time) / (vol*time**0.5)
    P = strike*torch.exp(-rate*time)*cdf(-d2) - stock*torch.exp(-dividend*time)*cdf(-d1)
    return P 

# baseline option value and 1% sensitivities - adjusting 1% deviation of the senstivties approximation
P = bs(stock, strike, rate, vol, time, dividend)
delta = bs(1.01*stock, strike, rate, vol, time, dividend)
rho = bs(stock, strike, 1.01*rate, vol, time, dividend)
vega = bs(stock, strike, rate, 1.01*vol, time, dividend)
theta = bs(stock, strike, rate, vol, 1.01*time, dividend)
epsilon = bs(stock, strike, rate, vol, time, 1.01*dividend)
strike_greek = bs(stock, 1.01*strike, rate, vol, time, dividend)

# scaling; in each case, δ is 1% of the starting value
delta_f = (delta - P) / (0.01*stock)
rho_f = (rho - P) / (0.01*rate)
vega_f = (vega - P) / (0.01*vol)
theta_f = (theta - P) / (0.01*time)
epsilon_f = (epsilon - P) / (0.01*dividend)
strike_greek_f = (strike_greek - P) / (0.01*strike)




In [ ]:
print(delta_f)
print(rho_f)
print(vega_f)
print(theta_f)
print(epsilon_f)
print(strike_greek_f)

tensor(-0.3907, grad_fn=<DivBackward0>)
tensor(-45.2968, grad_fn=<DivBackward0>)
tensor(38.4164, grad_fn=<DivBackward0>)
tensor(1.8803, grad_fn=<DivBackward0>)
tensor(40.2069, grad_fn=<DivBackward0>)
tensor(0.4654, grad_fn=<DivBackward0>)


In [ ]:
# Method 3
#Pytorch

# Once we have vector - value function we just call backward() to tell pytorch to calculate partial derivatives of our model. 
# because each input is tagged with require_grad = true and we can call .grad on each input to get its gradient

# Black Scholes- formula
d1 = (torch.log(stock/strike) + (rate - dividend + vol*vol/2)*time) / (vol*time**0.5)
d2 = (torch.log(stock/strike) + (rate - dividend - vol*vol/2)*time) / (vol*time**0.5)
P = strike*torch.exp(-rate*time)*cdf(-d2) - stock*torch.exp(-dividend*time)*cdf(-d1)

# run backpropagation 
P.backward() # performs the back prop computations

delta_b = stock.grad # calls the derivatives of out put with respect to chosen input 
rho_b = rate.grad
vega_b = vol.grad
theta_b = time.grad
epsilon_b = dividend.grad
strike_greek_b = strike.grad


In [ ]:
# Method 3
#Pytorch

# Once we have vector - value function we just call backward() to tell pytorch to calculate partial derivatives of our model. 
# because each input is tagged with require_grad = true and we can call .grad on each input to get its gradient

# Black Scholes- formula
d1 = (torch.log(stock/strike) + (rate - dividend + vol*vol/2)*time) / (vol*time**0.5)
d2 = (torch.log(stock/strike) + (rate - dividend - vol*vol/2)*time) / (vol*time**0.5)
P = strike*torch.exp(-rate*time)*cdf(-d2) - stock*torch.exp(-dividend*time)*cdf(-d1)

# run backpropagation 
P.backward() # performs the back prop computations

delta_b = stock.grad # calls the derivatives of out put with respect to chosen input 
rho_b = rate.grad
vega_b = vol.grad
theta_b = time.grad
epsilon_b = dividend.grad
strike_greek_b = strike.grad

In [ ]:
# Method 3
#Pytorch

# Once we have vector - value function we just call backward() to tell pytorch to calculate partial derivatives of our model. 
# because each input is tagged with require_grad = true and we can call .grad on each input to get its gradient

# Black Scholes- formula
d1 = (torch.log(stock/strike) + (rate - dividend + vol*vol/2)*time) / (vol*time**0.5)
d2 = (torch.log(stock/strike) + (rate - dividend - vol*vol/2)*time) / (vol*time**0.5)
P_1 = strike*torch.exp(-rate*time)*cdf(-d2) - stock*torch.exp(-dividend*time)*cdf(-d1)

# run backpropagation 
P_1.backward() # performs the back prop computations
# put x.grad.data.zero_() to reset the gradients so it does accumulate and go again https://discuss.pytorch.org/t/why-do-we-need-to-set-the-gradients-manually-to-zero-in-pytorch/4903/12 otherwise it accumlates gradients after each back pass. 
delta_b_1 = stock.grad # calls the derivatives of out put with respect to chosen input 
rho_b_1 = rate.grad
vega_b_1 = vol.grad
theta_b_1 = time.grad
epsilon_b_1 = dividend.grad
strike_greek_b_1 = strike.grad 

In [ ]:
print(delta_b_1,strike_greek_b_1,rho_b_1,theta_b_1,epsilon_b_1)

tensor(-0.4026) tensor(0.4535) tensor(-45.3458) tensor(1.8883) tensor(40.2573)


In [ ]:
torch.autograd.functional.jacobian(bs,(stock, strike, rate, vol, time, dividend))

(tensor(-0.4026),
 tensor(0.4535),
 tensor(-45.3458),
 tensor(38.4103),
 tensor(1.8883),
 tensor(40.2573))

Jacobian gives the same results as the direct greeks calculations 

In [ ]:
torch.autograd.functional.hessian(bs,(stock, strike, rate, vol, time, dividend))

((tensor(0.0240),
  tensor(-0.0240),
  tensor(2.4006),
  tensor(-0.1830),
  tensor(0.0494),
  tensor(-1.9981)),
 (tensor(-0.0240),
  tensor(0.0240),
  tensor(-2.8541),
  tensor(0.5672),
  tensor(-0.0305),
  tensor(2.4006)),
 (tensor(2.4006),
  tensor(-2.8541),
  tensor(285.4099),
  tensor(-56.7150),
  tensor(-42.2943),
  tensor(-240.0641)),
 (tensor(-0.1830),
  tensor(0.5672),
  tensor(-56.7152),
  tensor(4.3245),
  tensor(17.7491),
  tensor(18.3049)),
 (tensor(0.0494),
  tensor(-0.0305),
  tensor(-42.2943),
  tensor(17.7491),
  tensor(-1.5955),
  tensor(35.3176)),
 (tensor(-1.9981),
  tensor(2.4006),
  tensor(-240.0642),
  tensor(18.3049),
  tensor(35.3176),
  tensor(199.8068)))

In [ ]:
#Monte Carlo simulation of stock prices with automatic differentiation

# geometric brownian motion (single time step)
torch.manual_seed(42)
scenarios = 1000000
dW = vol * time**0.5 * torch.randn(size=(scenarios,))
r = torch.exp((rate - dividend - vol*vol/2) * time + dW)

# put option payoff & discounting
payoff = torch.max(strike - stock*r, torch.zeros(size=(scenarios,)))
ov = torch.mean(payoff) * torch.exp(-rate*time)

# run backpropagation to calculate option greeks
ov.backward()
delta = stock.grad
rho = rate.grad
vega = vol.grad
theta = time.grad
epsilon = dividend.grad
strike_greek = strike.grad
